# **From Google News API**

In [ ]:
#!pip install GoogleNews
!pip install --upgrade GoogleNews

     |████████████████████████████████| 286kB 16.5MB/s 


In [ ]:
!pip install newspaper3k

     |████████████████████████████████| 215kB 17.6MB/s 
     |████████████████████████████████| 81kB 13.5MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 7.4MB 27.2MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3358 sha256=00d538a923140f89b596721aedf76130ea81823c266dc3112b2c5b546108d0b0
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=e9223eef183ce9156abc3d28aaffacb36f0881998b5481c5f11690b84a8578b6
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=a4d82eb39569bb7362fbf42a6c2564f0a6944f34c3bcf035bdfce958a1f65b9e
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c3

In [ ]:
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
import pandas as pd
import nltk
googlenews = GoogleNews()

In [ ]:
googlenews.set_lang('en')
googlenews.get_news('Canada')

df=googlenews.results(sort=True)
df=pd.DataFrame(df)
df
df.to_excel("output.xlsx")  


In [ ]:
googlenews.clear()

# **Using News API**

In [ ]:
!pip install newsapi-python

In [ ]:
from newsapi import NewsApiClient
import datetime as dt
import pandas as pd

In [ ]:
newsapi = NewsApiClient(api_key='55b4edad61eb438584a451ef427f47b0')


In [ ]:
all_articles = newsapi.get_everything( q='walmart',from_param='2021-02-10',to='2021-02-11',language='en',page_size=100,sort_by='publishedAt')


In [ ]:
all_articles['totalResults']

587

In [ ]:
all_articles['articles'][0]

{'author': 'Elena DeBré',
 'content': 'Retail pharmacies in some states are now able to distribute the COVID vaccine through a program the Biden administration\xa0unveiled\xa0last week. Officials hope that the rollout at chains such as CVS and … [+6919 chars]',
 'description': 'The pharmacy chains were supposed to make national distribution easier. So far, they haven’t.',
 'publishedAt': '2021-02-11T23:37:16Z',
 'source': {'id': None, 'name': 'Slate Magazine'},
 'title': 'What’s Up With Coronavirus Vaccines and CVS?',
 'url': 'https://slate.com/technology/2021/02/coronavirus-vaccine-pharmacy-rollout.html',
 'urlToImage': 'https://compote.slate.com/images/0cce38e9-fab1-4a4d-a1d8-1b593877f425.jpeg?width=780&height=520&rect=780x520&offset=0x0'}

In [ ]:
new_list = all_articles['articles']


In [ ]:
for x,y in enumerate(new_list):
  print(f'{x} {y["title"]}') 

0 What’s Up With Coronavirus Vaccines and CVS?
1 Apple AirPods Pro are back down to $180 right now
2 Here's What Stood Out in Starbucks' Earnings Report
3 Inside CEW’s State of the Beauty Industry Report
4 ShopAtHome honoring Walmart+ deal….if you’re persistent
5 Biden administration pauses Trump's plans to ban WeChat, TikTok
6 No earbuds? No problem -- these stylish audio sunglasses sound good and are 20% off - CNET
7 SHAREHOLDER DEADLINE REMINDER: The Schall Law Firm Announces the Filing of a Class Action Lawsuit Against Walmart Inc. and Encourages Investors with Losses in Excess of $100,000 to Contact the Firm
8 Democrats favor jobs for Visa workers over U.S. graduates
9 Wonder Nation Girls' or Boys' Puffer Jackets (various colors) $9 + Free S/H w/ Walmart+ or FS on $35+
10 Biden Administration Requests Court Pause Trump Move to Ban WeChat
11 Presidents Day 2021 sales: Best deals on electronics and tech - lehighvalleylive.com
12 Presidents Day 2021 sales: Best deals on electronics a

In [ ]:
df=pd.DataFrame(new_list)
df.head(3)


,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'UN News'}",United Nations Headquarters,"Adopting 4 Resolutions, General Assembly Stres...","Against the backdrop of the COVID‑19 pandemic,...",https://www.un.org/press/en/2020/ga12301.doc.htm,None,2020-12-14T00:00:00Z,"Against the backdrop of the COVID19 pandemic, ..."
1,"{'id': None, 'name': 'Nature.com'}","Elena Ferri, Adrien Le Thomas, Heidi Ackerly W...",Activation of the IRE1 RNase through remodelin...,The RNase activity of Inositol-Requiring Enzym...,https://www.nature.com/articles/s41467-020-199...,https://media.springernature.com/m685/springer...,2020-12-14T00:00:00Z,"<li></li>\r\n1.Walter, P. &amp; Ron, D. The un..."
2,"{'id': None, 'name': 'Yahoo Entertainment'}",PR Newswire,New Industry Research Finds Companies That Use...,"Amplitude, the leader in Product Intelligence,...",https://finance.yahoo.com/news/industry-resear...,https://s.yimg.com/uu/api/res/1.2/kFcpiO31ZBAm...,2020-12-14T00:00:00Z,Global report from Amplitude exposes how busin...


In [ ]:
#df.to_excel("newsapi.xlsx")  
df.to_csv('11.csv',columns=['title','publishedAt'])


# Web-Scrapping

In [ ]:
import pandas as pd


In [ ]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    
    
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
        writer.writerows(articles)
        
    return articles

In [ ]:
articles = get_the_news('HomeDepot')


In [ ]:
articles[:]


In [ ]:
df = pd.DataFrame(articles, columns=['headline','source','posted','description','raw_link']) 
df

,headline,source,posted,description,raw_link
0,Home Depot Inc. stock outperforms market on st...,Market Watch,7 hours ago,Shares of Home Depot Inc. rose 1.21% to $258.3...,https://www.marketwatch.com/story/home-depot-i...
1,How to Trade Home Depot After It Reports Earnings,TheStreet.com,4 days ago,Get Report has been struggling ahead of earnin...,https://www.thestreet.com/investing/home-depot...
2,"Home Depot Tops Q4 Earnings Forecast, Holds Ba...",TheStreet.com,4 days ago,Get Report posted stronger-than-expected fourt...,https://www.thestreet.com/investing/home-depot...
3,"In year of pandemic, Home Depot became supplie...",San Francisco Chronicle,3 days ago,The housing market was among the very few brig...,https://www.sfgate.com/business/article/In-a-y...
4,Why Home Depot Stock Dropped 5% Today | The Mo...,The Motley Fool,3 days ago,Home Depot (NYSE:HD) reported another strong q...,https://www.fool.com/investing/2021/02/23/why-...
...,...,...,...,...,...
677,"US STOCKS-Tech selloff hits Nasdaq, Powell com...",Investing.com,4 days ago,"US STOCKS-Tech selloff hits Nasdaq, Powell com...",https://za.investing.com/news/us-stockstech-se...
678,"US STOCKS-Tech selloff hits Nasdaq, Powell com...",Investing.com,4 days ago,"US STOCKS-Tech selloff hits Nasdaq, Powell com...",https://ph.investing.com/news/stock-market-new...
679,Tech Stocks May Lead To Way Lower In Early Tra...,RTT News,4 days ago,After ending the previous session mostly lower...,https://www.rttnews.com/3172005/tech-stocks-ma...
680,Net Lease Real Estate ETF Brings Opportunity i...,Benzinga via Yahoo Finance,4 days ago,The Fundamental Income Net Lease Corporate Rea...,https://finance.yahoo.com/news/net-lease-real-...
